In [329]:
# https://preppindata.blogspot.com/2021/03/2021-week-9-working-with-strings.html

import pandas as pd
import numpy as np

Input the Customer Information file

In [330]:
df_customer = pd.read_excel(r'data/PD 2021 Wk 9 Input/Customer Information.xlsx', sheet_name = 'Sheet1')
df_customer

,IDs
0,"Ju856452,13S24-SPL wd234175,29M77-SPL KZ621372..."
1,"jM391563,00C69-SPL Uc296328,17S73-SBP EL580409..."
2,"rV469041,02L68-HS Rn519453,20L22-SPL pd615208,..."
3,"GQ505960,03W64-SBP JS186662,22M1-SBP Id680462,..."
4,"bf677129,05D99-SBP MA755072,24A76-SBP Zf805822..."
...,...
195,"nB634036,03W39-SBP Hg244254,22M64-SPL LD463660..."
196,"Ya795584,05D71-SPL TE924789,24A9-SBP rf761703,..."
197,"oi598659,06H99-HS zs924627,25D2-SPL XL027397,3..."
198,"JI221536,07M95-SBP hz193964,26S45-SBP gq663772..."


### split the values and reshape the data so there is a separate ID on each row

In [331]:

df_customer['IDs'] = df_customer['IDs'].astype(str).str.split(' ')
df_customer = df_customer.explode(['IDs'])
df_customer

,IDs
0,"Ju856452,13S24-SPL"
0,"wd234175,29M77-SPL"
0,"KZ621372,42K26-SBP"
0,"AY559207,53K50-HS"
1,"jM391563,00C69-SPL"
...,...
199,"Tf723726,09H15-SBP"
199,"et098928,28F45-SPL"
199,"Nr833098,41A65-HS"
199,"Md961094,52A25-SBP"


### Each ID field contains the following information we need to extract: (help)
- The first 6 digits present in each ID is the customers phone number
- The first 2 digits after the ‘,’ is the last 2 digits of the area code 
- The letter following this is the first letter of the name of the area that they are calling from
- The digits after this letter resemble the quantity of products ordered
- The letters after the ‘-‘ are the product ID codes 

In [332]:
df_customer[['phone number','code_last_2d','1st_Letter','quantity','product_ID']] = df_customer['IDs'].str.extract('.{2}(\d{6}),(\d{2})(\w)(\d*)-(.*)',expand=True)


### Rename these fields appropriately, and remove any unwanted columns – leaving only these 5 columns in the workflow. 

In [333]:
df_customer.drop(columns=['IDs'])

,phone number,code_last_2d,1st_Letter,quantity,product_ID
0,856452,13,S,24,SPL
0,234175,29,M,77,SPL
0,621372,42,K,26,SBP
0,559207,53,K,50,HS
1,391563,00,C,69,SPL
...,...,...,...,...,...
199,723726,09,H,15,SBP
199,098928,28,F,45,SPL
199,833098,41,A,65,HS
199,961094,52,A,25,SBP


### Input the Area Code Lookup Table – find a way to join it to the Customer information file

In [334]:
df_area_lookup = pd.read_excel(r'data/PD 2021 Wk 9 Input/Area Code Lookup.xlsx', sheet_name = 'Sheet1')
df_area_lookup['code_last_2d'] = df_area_lookup['Code'].astype(str).str[-2:]
df_area_lookup['1st_Letter'] = df_area_lookup['Area'].str.extract('(\w).*', expand=True)
df_area_lookup

,Code,Area,code_last_2d,1st_Letter
0,114,Sheffield,14,S
1,115,Nottingham,15,N
2,116,Leicester,16,L
3,117,Bristol,17,B
4,118,Reading,18,R
...,...,...,...,...
748,2894,Antrim,94,A
749,2895,Belfast,95,B
750,2896,Belfast,96,B
751,2897,Saintfield,97,S


In [335]:
df_customer_w_A = df_customer.merge(df_area_lookup, on =['1st_Letter','code_last_2d'], how='left')
df_customer_w_A

,IDs,phone number,code_last_2d,1st_Letter,quantity,product_ID,Code,Area
0,"Ju856452,13S24-SPL",856452,13,S,24,SPL,18513,Stornoway
1,"wd234175,29M77-SPL",234175,29,M,77,SPL,1629,Matlock
2,"KZ621372,42K26-SBP",621372,42,K,26,SBP,1542,Keith
3,"KZ621372,42K26-SBP",621372,42,K,26,SBP,2842,Kircubbin
4,"AY559207,53K50-HS",559207,53,K,50,HS,1553,Kings Lynn
...,...,...,...,...,...,...,...,...
1241,"et098928,28F45-SPL",098928,28,F,45,SPL,1328,Fakenham
1242,"Nr833098,41A65-HS",833098,41,A,65,HS,1241,Arbroath
1243,"Md961094,52A25-SBP",961094,52,A,25,SBP,1252,Aldershot
1244,"Md961094,52A25-SBP",961094,52,A,25,SBP,19752,Alford (Aberdeen)


### We don’t actually sell products in Clevedon, Fakenham, or Stornoway. Exclude these from our dataset

In [336]:
exclude_values = ['Clevedon', 'Fakenham', 'Stornoway']
df_customer_w_A = df_customer_w_A[~df_customer_w_A['Area'].isin(exclude_values)]
df_customer_w_A

,IDs,phone number,code_last_2d,1st_Letter,quantity,product_ID,Code,Area
1,"wd234175,29M77-SPL",234175,29,M,77,SPL,1629,Matlock
2,"KZ621372,42K26-SBP",621372,42,K,26,SBP,1542,Keith
3,"KZ621372,42K26-SBP",621372,42,K,26,SBP,2842,Kircubbin
4,"AY559207,53K50-HS",559207,53,K,50,HS,1553,Kings Lynn
5,"jM391563,00C69-SPL",391563,00,C,69,SPL,1200,Clitheroe
...,...,...,...,...,...,...,...,...
1240,"Tf723726,09H15-SBP",723726,09,H,15,SBP,1409,Holsworthy
1242,"Nr833098,41A65-HS",833098,41,A,65,HS,1241,Arbroath
1243,"Md961094,52A25-SBP",961094,52,A,25,SBP,1252,Aldershot
1244,"Md961094,52A25-SBP",961094,52,A,25,SBP,19752,Alford (Aberdeen)


### In some cases, the ID field does not provide accurate enough conditions to know where the customer is from. Exclude any phone numbers where the join has produced duplicated records.

In [337]:
df_customer_w_A = df_customer_w_A.drop_duplicates(subset=['phone number','quantity','product_ID','code_last_2d','1st_Letter'],keep=False)
df_customer_w_A

,IDs,phone number,code_last_2d,1st_Letter,quantity,product_ID,Code,Area
1,"wd234175,29M77-SPL",234175,29,M,77,SPL,1629,Matlock
4,"AY559207,53K50-HS",559207,53,K,50,HS,1553,Kings Lynn
8,"Uc296328,17S73-SBP",296328,17,S,73,SBP,1917,Sunderland
9,"EL580409,31E18-SBP",580409,31,E,18,SBP,131,Edinburgh
12,"Oi961467,54U1-SPL",961467,54,U,1,SPL,1854,Ullapool
...,...,...,...,...,...,...,...,...
1238,"Me752977,50C90-HS",752977,50,C,90,HS,1650,Cemmaes Road
1239,"AL414564,63A65-HS",414564,63,A,65,HS,1863,Ardgay
1240,"Tf723726,09H15-SBP",723726,09,H,15,SBP,1409,Holsworthy
1242,"Nr833098,41A65-HS",833098,41,A,65,HS,1241,Arbroath


### Remove any unwanted fields created from the join. 

In [338]:
df_customer_w_A = df_customer_w_A[['phone number','quantity','product_ID','Code','Area']]
df_customer_w_A

,phone number,quantity,product_ID,Code,Area
1,234175,77,SPL,1629,Matlock
4,559207,50,HS,1553,Kings Lynn
8,296328,73,SBP,1917,Sunderland
9,580409,18,SBP,131,Edinburgh
12,961467,1,SPL,1854,Ullapool
...,...,...,...,...,...
1238,752977,90,HS,1650,Cemmaes Road
1239,414564,65,HS,1863,Ardgay
1240,723726,15,SBP,1409,Holsworthy
1242,833098,65,HS,1241,Arbroath


### Join this dataset to our product lookup table. 

In [339]:
df_product_lookup = pd.read_excel(r'data/PD 2021 Wk 9 Input/Product Lookup.xlsx', sheet_name = 'Sheet1')
df_product_lookup
df_customer_w_AP = df_customer_w_A.merge(df_product_lookup,left_on='product_ID',right_on='Product ID',how='left')
df_customer_w_AP

,phone number,quantity,product_ID,Code,Area,Product ID,Product Name,Price
0,234175,77,SPL,1629,Matlock,SPL,"Soap, Liquid",£6.50
1,559207,50,HS,1553,Kings Lynn,HS,Hand Sanitiser,£2.29
2,296328,73,SBP,1917,Sunderland,SBP,"Soap, Bar",£4.55
3,580409,18,SBP,131,Edinburgh,SBP,"Soap, Bar",£4.55
4,961467,1,SPL,1854,Ullapool,SPL,"Soap, Liquid",£6.50
...,...,...,...,...,...,...,...,...
746,752977,90,HS,1650,Cemmaes Road,HS,Hand Sanitiser,£2.29
747,414564,65,HS,1863,Ardgay,HS,Hand Sanitiser,£2.29
748,723726,15,SBP,1409,Holsworthy,SBP,"Soap, Bar",£4.55
749,833098,65,HS,1241,Arbroath,HS,Hand Sanitiser,£2.29


### For each area, and product, find the total sales values, rounded to zero decimal places

In [340]:
# check datatype
df_customer_w_AP.dtypes

phone number    object
quantity        object
product_ID      object
Code             int64
Area            object
Product ID      object
Product Name    object
Price           object
dtype: object

In [341]:
df_customer_w_AP['quantity'] = df_customer_w_AP['quantity'].astype(int)
df_customer_w_AP['Price'] = df_customer_w_AP['Price'].str[1:].astype(float)
df_customer_w_AP['Revenue'] = df_customer_w_AP['quantity'] * df_customer_w_AP['Price']


df_agg = df_customer_w_AP.groupby(['Area','Product Name'])['Revenue'].sum().reset_index()
df_agg['Revenue'] = df_agg['Revenue'].round(0)
df_agg


,Area,Product Name,Revenue
0,Aberdeen,Hand Sanitiser,353.0
1,Aberdeen,"Soap, Bar",2507.0
2,Aberdeen,"Soap, Liquid",793.0
3,Arbroath,Hand Sanitiser,847.0
4,Arbroath,"Soap, Bar",687.0
...,...,...,...
139,Worthing,"Soap, Bar",2020.0
140,Worthing,"Soap, Liquid",1131.0
141,Wrexham,Hand Sanitiser,545.0
142,Wrexham,"Soap, Bar",2129.0


### Rank how well each product sold in each area

In [342]:
df_agg['rank'] = df_agg.groupby(['Area'])['Revenue'].rank(ascending=False).astype(int)


### For each area, work out the percent of total that each different product contributes to the overall revenue of that Area, rounded to 2 decimal places.

In [343]:
df_agg['overall_revenue'] = df_agg.groupby(['Area'])['Revenue'].transform('sum')
df_agg['% of total - Product'] = df_agg['Revenue']/df_agg['overall_revenue']*100
df_agg['% of total - Product'] = df_agg['% of total - Product'].round(2)
df_agg

,Area,Product Name,Revenue,rank,overall_revenue,% of total - Product
0,Aberdeen,Hand Sanitiser,353.0,3,3653.0,9.66
1,Aberdeen,"Soap, Bar",2507.0,1,3653.0,68.63
2,Aberdeen,"Soap, Liquid",793.0,2,3653.0,21.71
3,Arbroath,Hand Sanitiser,847.0,2,3958.0,21.40
4,Arbroath,"Soap, Bar",687.0,3,3958.0,17.36
...,...,...,...,...,...,...
139,Worthing,"Soap, Bar",2020.0,1,3485.0,57.96
140,Worthing,"Soap, Liquid",1131.0,2,3485.0,32.45
141,Wrexham,Hand Sanitiser,545.0,3,3337.0,16.33
142,Wrexham,"Soap, Bar",2129.0,1,3337.0,63.80


### Output the data

In [344]:
# re-order and select the only needed column
df_agg = df_agg[['rank','Area','Product Name','Revenue','% of total - Product']]
df_agg.sort_values(['Area','rank']).reset_index(drop='index')

df_agg.to_csv(r'output/2021-week9-output.csv')